In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import torch
import re
from sklearn.metrics import balanced_accuracy_score, roc_auc_score,accuracy_score
from Constants import *
from Preprocessing import *
from Models import *
from Utils import *
pd.set_option('display.max_rows', 200)



In [3]:
pd.read_csv('../data/digital_twin_ln_monograms.csv')

,id,L1A,L1B,L2A,L2B,L3,L4,L5A,L5B,L6,...,R1A,R1B,R2A,R2B,R3,R4,R5A,R5B,R6,RRPLN
0,3,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,6,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
3,7,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
531,10201,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
532,10202,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
533,10203,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
534,10204,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0


In [8]:
data = DTDataset(use_smote=True)
data.processed_df.T
# data.processed_df#.shape, len(data.processed_df.index.unique())

/data/DigitalTwinVis/python/Preprocessing.py:252: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_file)
/usr/local/lib/python3.8/dist-packages/imblearn/over_sampling/_smote/base.py:345: FutureWarning: The parameter `n_jobs` has been deprecated in 0.10 and will be removed in 0.12. You can pass an nearest neighbors estimator where `n_jobs` is already set instead.
  warnings.warn(


id,3,5,6,7,8,9,10,11,13,14,...,10196,10197,10198,10199,10200,10201,10202,10203,10204,10205
hpv,1,0,1,1,1,1,-1,1,0,1,...,0,1,-1,0,1,1,0,1,0,1
age,55.969444,20.95,69.930556,72.319444,59.730556,60.083333,67.708333,57.858333,51.758333,56.25,...,47.619444,50.163889,70.888889,67.825,56.336111,49.566667,48.705556,77.116667,45.95,49.733333
packs_per_year,0.0,38.0,35.0,0.0,0.0,0.0,40.0,44.0,0.0,40.0,...,5.0,0.0,50.0,0.0,0.0,30.0,30.0,0.0,5.0,0.0
smoking_status,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,...,0.5,0.0,0.5,0.0,0.0,1.0,1.0,0.0,0.5,0.0
gender,1,1,0,1,1,1,1,1,1,1,...,0,1,0,1,1,1,1,1,1,1
Aspiration rate Pre-therapy,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
total_dose,66.0,72.0,70.0,70.0,66.0,66.0,69.96,70.0,70.0,70.0,...,70.0,72.0,66.0,70.0,69.96,70.0,72.0,70.0,69.96,69.96
dose_fraction,2.2,1.8,2.121212,2.121212,2.2,2.2,2.12,2.121212,2.0,2.121212,...,2.121212,1.8,2.2,2.121212,2.12,2.121212,1.714286,2.333333,2.12,2.12
bilateral,False,False,True,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,True,False
White/Caucasion,True,True,True,True,False,True,True,True,True,True,...,True,True,True,True,True,True,False,True,True,True


In [9]:
get_tt_split()[0]

/data/DigitalTwinVis/python/Preprocessing.py:192: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


[5,
 6,
 8,
 11,
 13,
 14,
 15,
 16,
 17,
 18,
 21,
 23,
 24,
 26,
 27,
 28,
 32,
 33,
 37,
 38,
 39,
 40,
 41,
 42,
 48,
 49,
 50,
 51,
 53,
 55,
 56,
 57,
 60,
 64,
 65,
 67,
 69,
 71,
 74,
 75,
 78,
 79,
 80,
 81,
 82,
 87,
 88,
 91,
 94,
 96,
 99,
 103,
 109,
 116,
 119,
 120,
 121,
 125,
 148,
 150,
 153,
 178,
 181,
 183,
 185,
 186,
 188,
 191,
 192,
 193,
 196,
 197,
 198,
 200,
 201,
 203,
 204,
 205,
 206,
 207,
 210,
 212,
 213,
 214,
 216,
 218,
 219,
 220,
 221,
 222,
 223,
 225,
 226,
 229,
 230,
 231,
 232,
 233,
 234,
 235,
 237,
 238,
 239,
 240,
 241,
 243,
 244,
 246,
 247,
 248,
 249,
 251,
 252,
 253,
 255,
 256,
 257,
 258,
 259,
 260,
 261,
 262,
 263,
 265,
 266,
 269,
 270,
 273,
 275,
 276,
 277,
 278,
 280,
 281,
 282,
 283,
 285,
 289,
 2000,
 2002,
 2003,
 2004,
 2007,
 2008,
 2009,
 2010,
 2011,
 2012,
 2013,
 2014,
 2016,
 2018,
 2021,
 2022,
 2023,
 2025,
 2027,
 2028,
 2030,
 2033,
 5000,
 5002,
 5004,
 5005,
 5006,
 5008,
 5009,
 5010,
 5011,
 5012,
 5

In [10]:
get_tt_split(resample_training=True)[0]

/data/DigitalTwinVis/python/Preprocessing.py:192: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


array([10197, 10003,  5010, 10147,  5090,  5087,  5089,   121,   119,
       10036,  5068, 10082, 10048, 10019, 10150, 10159,  5070,    28,
         205, 10054, 10041, 10073, 10098,  2014,  5102, 10024, 10198,
        2010,   248,  2013,   233, 10189,   289, 10114,    17,   237,
        2025,   230,  2013,  2021,  5071,  2033, 10077,  5068, 10090,
         238,   230,    80,   270,  5004,  2033, 10182,  5051,    28,
          14,   270,  2016,  2022,  5095,    78,  5025,   246, 10201,
       10006,    21,   243,  5000, 10188, 10164,   121, 10042,    26,
       10116, 10064, 10119,   198,  5081, 10198, 10064,   188,   252,
        2016, 10036, 10087, 10083,  5063,   260,  2016,   244,    99,
         206, 10129,  5000,  5091,    37, 10054, 10059,   214,   269,
       10087,   119,    53,  5043,   233,  5004, 10152,    24, 10033,
       10038, 10115,   263,  5085,  5119, 10018,   248,    15,  5016,
       10081,  5026,   265,   198, 10034,   205,  5091,   266,  5106,
       10074,   153,

In [11]:
def nllloss(ytrue,ypred):
    #nll loss with argmax added in
    loss = torch.nn.NLLLoss()
    return loss(ypred,ytrue.argmax(axis=1))

def state_loss(ytrue,ypred,weights=[1,1,1,1]):
    pd_loss = nllloss(ytrue[0],ypred[0])*weights[0]
    nd_loss = nllloss(ytrue[1],ypred[1])*weights[1]
    mod_loss = nllloss(ytrue[2],ypred[2])*weights[2]
    loss = pd_loss + nd_loss + mod_loss
    dlt_true = ytrue[3]
    dlt_pred = ypred[3]
    ndlt = dlt_true.shape[1]
#     nloss = torch.nn.NLLLoss()
    bce = torch.nn.BCELoss()
    for i in range(ndlt):
        dlt_loss = bce(dlt_pred[:,i].view(-1),dlt_true[:,i].view(-1))
        loss += dlt_loss*weights[3]/ndlt
    return loss

def outcome_loss(ytrue,ypred,weights=[1,1,1]):
    loss = 0
    nloss = torch.nn.BCELoss()
    for i in range(len(weights)):
        iloss = nloss(ypred[:,i],ytrue[i])*weights[i]
        loss += iloss
    return loss

def mc_metrics(yt,yp,numpy=False,is_dlt=False):
    if not numpy:
        yt = yt .cpu().detach().numpy()
        yp = yp.cpu().detach().numpy()
    #dlt prediction (binary)
    if is_dlt:
        acc = accuracy_score(yt,yp>.5)
        if yt.sum() > 1:
            auc = roc_auc_score(yt,yp)
        else:
            auc=-1
        error = np.mean((yt-yp)**2)
        return {'accuracy': acc, 'mse': error, 'auc': auc}
    #this is a catch for when I se the dlt prediction format (encoded integer ordinal, predict as a categorical and take the argmax)
    elif yt.ndim > 1:
        try:
            bacc = balanced_accuracy_score(yt.argmax(axis=1),yp.argmax(axis=1))
        except:
            bacc = -1
        try:
            roc_micro = roc_auc_score(yt,yp,average='micro')
        except:
            roc_micro=-1
        try:
            roc_macro = roc_auc_score(yt,yp,average='macro')
        except:
            roc_macro = -1
        return {'accuracy': bacc, 'roc_micro': roc_micro,'roc_macro': roc_macro}
    #outcomes (binary)
    else:
        if yp.ndim > 1:
            yp = yp.argmax(axis=1)
        try:
            bacc = accuracy_score(yt,yp)
        except:
            bacc = -1
        try:
            roc = roc_auc_score(yt,yp)
        except:
            roc = -1
        error = np.mean((yt-yp)**2)
        return {'accuracy': bacc, 'mse': error, 'auc': roc}

def state_metrics(ytrue,ypred,numpy=False):
    pd_metrics = mc_metrics(ytrue[0],ypred[0],numpy=numpy)
    nd_metrics = mc_metrics(ytrue[1],ypred[1],numpy=numpy)
    mod_metrics = mc_metrics(ytrue[1],ypred[1],numpy=numpy)
    
    dlt_metrics = []
    dlt_true = ytrue[3]
    dlt_pred = ypred[3]
    ndlt = dlt_true.shape[1]
    nloss = torch.nn.NLLLoss()
    for i in range(ndlt):
        dm = mc_metrics(dlt_true[:,i],dlt_pred[:,i].view(-1),is_dlt=True)
        dlt_metrics.append(dm)
    dlt_acc =[d['accuracy'] for d in dlt_metrics]
    dlt_error = [d['mse'] for d in dlt_metrics]
    dlt_auc = [d['auc'] for d in dlt_metrics]
    
    acc_mean = np.mean([a for a in dlt_acc if a >= 0])
    auc_mean = np.mean([a for a in dlt_auc if a >= 0])
    results = {'pd': pd_metrics,'nd': nd_metrics,'mod': mod_metrics,
               'dlts': {'accuracy': dlt_acc,'accuracy_mean': acc_mean,'auc': dlt_auc,'auc_mean': auc_mean}
              }
    return results

def outcome_metrics(ytrue,ypred,numpy=False):
    res = {}
    for i, outcome in enumerate(Const.outcomes):
        metrics = mc_metrics(ytrue[i],ypred[:,i])
        res[outcome] = metrics
    return res

def transition_sample(state):
    ids = get_dt_ids()
    
    train_ids, test_ids = get_tt_split()
    
    dataset = DTDataset()
    
    #only train on people with  IC for state 1 since other people can't have any outcomes otherwise
    require = None
    if state == 1:
        require = Const.decisions[0] #we don't expect a state update if there is no treatment
        valid_ids = dataset.get_input_state(require=require).index.values
        train_ids = [t for t in train_ids if t in valid_ids]
        test_ids = [t for t in test_ids if t in valid_ids]
    xtrain = dataset.get_input_state(step=state,ids=train_ids,require=require)
    xtest = dataset.get_input_state(step=state,ids=test_ids,require=require)
    ytrain = dataset.get_intermediate_outcomes(step=state,ids=train_ids,require=require)
    ytest = dataset.get_intermediate_outcomes(step=state,ids=test_ids,require=require)

    xtrain = df_to_torch(xtrain)
    xtest = df_to_torch(xtest)
    ytrain = [df_to_torch(t) for t in ytrain]
    ytest= [df_to_torch(t) for t in ytest]
    return xtrain,xtest,ytrain,ytest

In [12]:
def train_state(model=None,
                model_args={},
                state=1,
                split=.7,
                lr=.0001,
                epochs=1000,
                patience=10,
                use_attention=False,
                weights=None,
                save_path='../data/models/',
                use_default_split=True,
                use_bagging_split=False,
                resample_training=False,#use bootstraping on training data after splitting
                n_validation_trainsteps=2,
                verbose=True,
                use_smote=True,
                file_suffix=''):
    
    ids = get_dt_ids()
    
    train_ids, test_ids = get_tt_split(use_default_split=use_default_split,use_bagging_split=use_bagging_split,resample_training=resample_training)

    if use_smote:
        dataset = DTDataset(use_smote=True,smote_ids = train_ids)
        train_ids = [i for i in dataset.processed_df.index.values if i not in test_ids]
    else:
        dataset = DTDataset()
    
    #only train on people with  IC for state 1 since other people can't have any outcomes otherwise
    require = None
    if state == 1:
        require = Const.decisions[0] #we don't expect a state update if there is no treatment
        valid_ids = dataset.get_input_state(require=require).index.values
        train_ids = [t for t in train_ids if t in valid_ids]
        test_ids = [t for t in test_ids if t in valid_ids]
    xtrain = dataset.get_input_state(step=state,ids=train_ids,require=require)
    xtest = dataset.get_input_state(step=state,ids=test_ids,require=require)
    ytrain = dataset.get_intermediate_outcomes(step=state,ids=train_ids,require=require)
    ytest = dataset.get_intermediate_outcomes(step=state,ids=test_ids,require=require)
    

    if not use_attention:
        model_args = {k:v for k,v in model_args.items() if 'attention' not in k and 'embed_size' not in k}
    if state < 3:
        if model is None:
            if use_attention:
                model = OutcomeAttentionSimulator(xtrain.shape[1],state=state,**model_args)
            else:
                model = OutcomeSimulator(xtrain.shape[1],state=state,**model_args)
        lfunc = state_loss
        if weights is None:
            weights = [1,1,1,1]
    else:
        if model is None:
            if use_attention:
                model = EndpointAttentionSimulator(xtrain.shape[1],**model_args)
            else:
                model = EndpointSimulator(xtrain.shape[1],**model_args)
        if weights is None:
            weights = [1,1,1,1]
        lfunc = outcome_loss
        
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model.set_device(device)
    
    hashcode = str(hash(','.join([str(i) for i in train_ids])))
    save_file = save_path + 'model_' + model.identifier + '_split' + str(split) + '_resample' + str(resample_training) +  '_hash' + hashcode + file_suffix + '.tar'
    xtrain = df_to_torch(xtrain).to(device)
    xtest = df_to_torch(xtest).to(device)
    ytrain = [df_to_torch(t).to(device) for t in ytrain]
    ytest= [df_to_torch(t).to(device) for t in ytest]
    
    model.fit_normalizer(xtrain)
#     normalize = lambda x: (x - xtrain.mean(axis=0)+.01)/(xtrain.std(axis=0)+.01)
#     unnormalize = lambda x: (x * (xtrain.std(axis=0) +.01)) + xtrain.mean(axis=0) - .01
    
    optimizer = torch.optim.Adam(model.parameters(),lr=lr)
    best_val_loss = 1000000000000000000000000000
    best_loss_metrics = {}
    last_epoch = False
    for epoch in range(epochs):
        
        model.train(True)
        optimizer.zero_grad()
        
        xtrain_sample = xtrain#[torch.randint(len(xtrain),(len(xtrain),) )]
        ypred = model(xtrain_sample)
        loss = lfunc(ytrain,ypred,weights=weights)

        loss.backward()
        optimizer.step()
        if verbose:
            print('epoch',epoch,'train loss',loss.item())
        
        model.eval()
        yval = model(xtest)
        val_loss = lfunc(ytest,yval,weights=weights)
        if state < 3:
            val_metrics = state_metrics(ytest,yval)
        else:
            val_metrics = outcome_metrics(ytest,yval)
        if val_loss.item() < best_val_loss:
            best_val_loss = val_loss.item()
            best_loss_metrics = val_metrics
            steps_since_improvement = 0
            torch.save(model.state_dict(),save_file)
        else:
            steps_since_improvement += 1
        if verbose:
            print('val loss',val_loss.item())
            print('______________')
        if steps_since_improvement > patience:
            break
    print('best loss',best_val_loss,best_loss_metrics)
    model.load_state_dict(torch.load(save_file))
    
    #train one step on validation data
    for i in range(n_validation_trainsteps):
        model.train()
        yval = model(xtest)
        val_loss = lfunc(ytest,yval,weights=weights)
        val_loss.backward()
        optimizer.step()
        torch.save(model.state_dict(),save_file)
    
    model.eval()
    return model,  best_val_loss, best_loss_metrics

In [13]:
def train_model_ensemble(model_arg_list,state,
                         n_bags=10,
                         smote_options=[True,False],
                         smote_baseline_options=[True,False],
                         weight_options=[None],
                         weight_baseline_options=[None],
                         **kwargs):
    resampled_models =[]
    base_models = []
    base_metrics = []
    resample_metrics = []
    base_losses = []
    resample_losses = []
    n_errors = 0
    for margs in model_arg_list:
        for s in smote_baseline_options:
            for w in weight_baseline_options:
                [base_model,blosses,bmetrics] = train_state(model_args=margs,state=state,
                                         resample_training=False,
                                         use_smote=smote_options,
                                         weights=w,
                                         verbose=False,**kwargs)
            base_models.append(base_model)
            base_metrics.append(bmetrics)
            base_losses.append(blosses)
        for n in range(n_bags):
            for s in smote_options:
                for w in weight_options:
                    #this can fail if I resample a bad distribution with no minority classes
                    done = False
                    while not done:
                        try:
                            [model,loss,metrics] = train_state(model_args=margs,
                                                state=state,resample_training=True,
                                                use_smote=s,
                                                weights=w,
                                                verbose=False,**kwargs)
                            resampled_models.append(model)
                            resample_metrics.append(metrics)
                            resample_losses.append(loss)
                            done=True
                            print('model_done',len(base_models)+len(resampled_models))
                        except Exception as e:
                            print('error training model')
                            print(e)
                            n_errors += 1
    print('done with',n_errors,'errors')
    return base_models,resampled_models

In [14]:
t1_arglist = [
     {'hidden_layers': [400,400], 'dropout': 0.9, 'input_dropout': 0.1},
    {'hidden_layers': [400], 'dropout': .7, 'input_dropout': .2}
]
m1_list, m1_ensemble = train_model_ensemble(t1_arglist,1,
                                            n_bags=5,
                                            weight_options=[[1,1,1,10],[2,2,0.1,1]],
                                            weight_baseline_options=[[1,1,1,10],[2,2,0.1,1]],
                                           )
from Models import *
emodel1 = TransitionEnsemble(m1_list,m1_ensemble)
test2 = emodel1(transition_sample(1)[0][0].view(1,-1).to(emodel1.get_device()))
test2

/data/DigitalTwinVis/python/Preprocessing.py:192: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/data/DigitalTwinVis/python/Preprocessing.py:192: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/data/DigitalTwinVis/python/Preprocessing.py:252: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_file)


here


/usr/local/lib/python3.8/dist-packages/imblearn/over_sampling/_smote/base.py:345: FutureWarning: The parameter `n_jobs` has been deprecated in 0.10 and will be removed in 0.12. You can pass an nearest neighbors estimator where `n_jobs` is already set instead.
  warnings.warn(


best loss 5.831185340881348 {'pd': {'accuracy': 0.3670634920634921, 'roc_micro': 0.7557024793388429, 'roc_macro': 0.57798752714945}, 'nd': {'accuracy': 0.3333333333333333, 'roc_micro': 0.86455525606469, 'roc_macro': 0.49825292910399294}, 'mod': {'accuracy': 0.3333333333333333, 'roc_micro': 0.86455525606469, 'roc_macro': 0.49825292910399294}, 'dlts': {'accuracy': [0.7818181818181819, 0.9090909090909091, 1.0, 0.8, 1.0, 0.8909090909090909, 1.0, 0.9454545454545454], 'accuracy_mean': 0.9159090909090909, 'auc': [0.5658914728682171, 0.544, -1, 0.5433884297520661, -1, 0.41496598639455784, -1, 0.32692307692307687], 'auc_mean': 0.4790337931875836}}


/data/DigitalTwinVis/python/Preprocessing.py:192: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/data/DigitalTwinVis/python/Preprocessing.py:192: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/data/DigitalTwinVis/python/Preprocessing.py:252: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_file)


...

best loss 9.11441421508789 {'pd': {'accuracy': 0.29559748427672955, 'roc_micro': 0.6204938271604938, 'roc_macro': 0.43351737768025406}, 'nd': {'accuracy': 0.3333333333333333, 'roc_micro': 0.8701687307788214, 'roc_macro': 0.5055049911708048}, 'mod': {'accuracy': 0.3333333333333333, 'roc_micro': 0.8701687307788214, 'roc_macro': 0.5055049911708048}, 'dlts': {'accuracy': [0.8288288288288288, 0.918918918918919, 0.9819819819819819, 0.7747747747747747, 0.990990990990991, 0.8648648648648649, 0.990990990990991, 0.963963963963964], 'accuracy_mean': 0.9144144144144144, 'auc': [0.4765446224256293, 0.6013071895424837, 0.17431192660550454, 0.6162790697674418, -1, 0.4895833333333333, -1, 0.5560747663551402], 'auc_mean': 0.4856834846715888}}
model_done 43


/data/DigitalTwinVis/python/Preprocessing.py:192: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/data/DigitalTwinVis/python/Preprocessing.py:192: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/data/DigitalTwinVis/python/Preprocessing.py:252: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_file)


best loss 3.996500253677368 {'pd': {'accuracy': 0.4137537993920973, 'roc_micro': 0.7664517861939296, 'roc_macro': 0.5379005189235082}, 'nd': {'accuracy': 0.3333333333333333, 'roc_micro': 0.8660407239819004, 'roc_macro': 0.42043914838453955}, 'mod': {'accuracy': 0.3333333333333333, 'roc_micro': 0.8660407239819004, 'roc_macro': 0.42043914838453955}, 'dlts': {'accuracy': [0.8504672897196262, 0.9158878504672897, 1.0, 0.7850467289719626, 0.9906542056074766, 0.8691588785046729, 0.9813084112149533, 0.9345794392523364], 'accuracy_mean': 0.9158878504672897, 'auc': [0.5645604395604396, 0.5839002267573696, -1, 0.4192546583850932, -1, 0.49923195084485406, 0.6619047619047619, 0.3771428571428571], 'auc_mean': 0.5176658157658959}}
model_done 44
done with 2 errors


/data/DigitalTwinVis/python/Preprocessing.py:192: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/data/DigitalTwinVis/python/Preprocessing.py:192: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/data/DigitalTwinVis/python/Preprocessing.py:252: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_file)


{'predictions': [tensor([[0.1601, 0.8032, 0.0367]], device='cuda:0', grad_fn=<MeanBackward1>),
  tensor([[0.1200, 0.8374, 0.0426]], device='cuda:0', grad_fn=<MeanBackward1>),
  tensor([[0.6648, 0.0523, 0.0521, 0.0558, 0.0516, 0.0728, 0.0505]],
         device='cuda:0', grad_fn=<MeanBackward1>),
  tensor([[0.1356, 0.1070, 0.1103, 0.0981, 0.0763, 0.1087, 0.0749, 0.0937]],
         device='cuda:0', grad_fn=<MeanBackward1>)],
 '5%': [tensor([[0.1030, 0.3959, 0.0268]], device='cuda:0', grad_fn=<ViewBackward0>),
  tensor([[0.0324, 0.6425, 0.0240]], device='cuda:0', grad_fn=<ViewBackward0>),
  tensor([[0.2498, 0.0251, 0.0205, 0.0268, 0.0186, 0.0462, 0.0179]],
         device='cuda:0', grad_fn=<ViewBackward0>),
  tensor([[0.0525, 0.0626, 0.0202, 0.0595, 0.0230, 0.0615, 0.0181, 0.0298]],
         device='cuda:0', grad_fn=<ViewBackward0>)],
 '95%': [tensor([[0.4114, 0.8670, 0.2000]], device='cuda:0', grad_fn=<ViewBackward0>),
  tensor([[0.1815, 0.9454, 0.1818]], device='cuda:0', grad_fn=<ViewBac

In [15]:
t2_arglist = [
     {'hidden_layers': [400,400], 'dropout': 0.9, 'input_dropout': 0.1},
    {'hidden_layers': [400], 'dropout': .7, 'input_dropout': .2}
]
m2_list, m2_ensemble = train_model_ensemble(t2_arglist,2,
                                            weight_options=[[1,1,1,10],[2,2,0.1,1]],
                                            weight_baseline_options=[[1,1,1,10],[2,2,0.1,1]],
                                            n_bags=5)
emodel2 = TransitionEnsemble(m2_list,m2_ensemble)
emodel2(transition_sample(2)[0][0].view(1,-1))

/data/DigitalTwinVis/python/Preprocessing.py:192: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/data/DigitalTwinVis/python/Preprocessing.py:192: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/data/DigitalTwinVis/python/Preprocessing.py:252: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_file)


here


/usr/local/lib/python3.8/dist-packages/imblearn/over_sampling/_smote/base.py:345: FutureWarning: The parameter `n_jobs` has been deprecated in 0.10 and will be removed in 0.12. You can pass an nearest neighbors estimator where `n_jobs` is already set instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:2184: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:2184: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:2184: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:2184: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred cont

best loss 3.271223545074463 {'pd': {'accuracy': 0.5, 'roc_micro': 0.9432025601517127, 'roc_macro': -1}, 'nd': {'accuracy': 0.3482990506329114, 'roc_micro': 0.72885637977912, 'roc_macro': 0.5228555990751113}, 'mod': {'accuracy': 0.3482990506329114, 'roc_micro': 0.72885637977912, 'roc_macro': 0.5228555990751113}, 'dlts': {'accuracy': [0.952054794520548, 0.9657534246575342, 1.0, 0.9178082191780822, 1.0, 0.9657534246575342, 1.0, 0.9794520547945206], 'accuracy_mean': 0.9726027397260275, 'auc': [0.7307297019527236, 0.4879432624113475, -1, 0.6791044776119404, -1, 0.597163120567376, -1, 0.5477855477855478], 'auc_mean': 0.6085452220657871}}


/data/DigitalTwinVis/python/Preprocessing.py:192: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/data/DigitalTwinVis/python/Preprocessing.py:192: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/data/DigitalTwinVis/python/Preprocessing.py:252: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_file)


...

best loss 3.5472054481506348 {'pd': {'accuracy': 0.33959675495108566, 'roc_micro': 0.9198282961609189, 'roc_macro': 0.5198520081867114}, 'nd': {'accuracy': 0.36734693877551017, 'roc_micro': 0.7631424763917928, 'roc_macro': 0.5737803483616369}, 'mod': {'accuracy': 0.36734693877551017, 'roc_micro': 0.7631424763917928, 'roc_macro': 0.5737803483616369}, 'dlts': {'accuracy': [0.959866220735786, 0.9665551839464883, 1.0, 0.9364548494983278, 0.9933110367892977, 0.9665551839464883, 0.9966555183946488, 0.9765886287625418], 'accuracy_mean': 0.9744983277591973, 'auc': [0.7540650406504065, 0.4816608996539792, -1, 0.7097744360902256, 0.5858585858585859, 0.6363321799307959, -1, 0.5210371819960861], 'auc_mean': 0.6147880540300131}}
model_done 43


/data/DigitalTwinVis/python/Preprocessing.py:192: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/data/DigitalTwinVis/python/Preprocessing.py:192: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/data/DigitalTwinVis/python/Preprocessing.py:252: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_file)
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:2184: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:2184: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:2184: UserWarning

best loss 2.802062749862671 {'pd': {'accuracy': 0.5119047619047619, 'roc_micro': 0.9390916106272738, 'roc_macro': -1}, 'nd': {'accuracy': 0.36277676828857935, 'roc_micro': 0.7525167576187984, 'roc_macro': 0.5399430391630767}, 'mod': {'accuracy': 0.36277676828857935, 'roc_micro': 0.7525167576187984, 'roc_macro': 0.5399430391630767}, 'dlts': {'accuracy': [0.9547038327526133, 0.9790940766550522, 1.0, 0.9372822299651568, 0.9965156794425087, 0.9721254355400697, 1.0, 0.9825783972125436], 'accuracy_mean': 0.977787456445993, 'auc': [0.5833801235261089, 0.47271648873072364, -1, 0.6491119372160264, -1, 0.6621863799283154, -1, 0.6510638297872341], 'auc_mean': 0.6036917518376818}}
model_done 44
done with 5 errors


/data/DigitalTwinVis/python/Preprocessing.py:192: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/data/DigitalTwinVis/python/Preprocessing.py:192: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/data/DigitalTwinVis/python/Preprocessing.py:252: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_file)


{'predictions': [tensor([[0.7433, 0.2088, 0.0478]], device='cuda:0', grad_fn=<MeanBackward1>),
  tensor([[0.3941, 0.5293, 0.0766]], device='cuda:0', grad_fn=<MeanBackward1>),
  tensor([[0.2794, 0.5112, 0.1177, 0.0918]], device='cuda:0',
         grad_fn=<MeanBackward1>),
  tensor([[0.0995, 0.0975, 0.0781, 0.0997, 0.0895, 0.0898, 0.1000, 0.0930]],
         device='cuda:0', grad_fn=<MeanBackward1>)],
 '5%': [tensor([[0.6813, 0.0785, 0.0194]], device='cuda:0', grad_fn=<ViewBackward0>),
  tensor([[0.2503, 0.3288, 0.0232]], device='cuda:0', grad_fn=<ViewBackward0>),
  tensor([[0.0146, 0.4028, 0.0414, 0.0118]], device='cuda:0',
         grad_fn=<ViewBackward0>),
  tensor([[0.0180, 0.0187, 0.0024, 0.0150, 0.0087, 0.0098, 0.0028, 0.0107]],
         device='cuda:0', grad_fn=<ViewBackward0>)],
 '95%': [tensor([[0.8892, 0.2548, 0.0849]], device='cuda:0', grad_fn=<ViewBackward0>),
  tensor([[0.6064, 0.7138, 0.0814]], device='cuda:0', grad_fn=<ViewBackward0>),
  tensor([[0.3128, 0.9321, 0.2114, 0.1

In [16]:
t3_arglist = [
     {'hidden_layers': [50], 'dropout': 0.5, 'input_dropout': 0},
]
m3_list, m3_ensemble = train_model_ensemble(t3_arglist,3,n_bags=5,
                                            weight_options=[[1,1,1,1],[1,2,2,1]],
                                            weight_baseline_options=[[1,1,1,1],[1,2,2,1]],
                                           )
emodel3 = EndpointEnsemble(m3_list,m3_ensemble)
emodel3(transition_sample(3)[0][0].view(1,-1))

/data/DigitalTwinVis/python/Preprocessing.py:192: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/data/DigitalTwinVis/python/Preprocessing.py:192: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/data/DigitalTwinVis/python/Preprocessing.py:252: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_file)


here


/usr/local/lib/python3.8/dist-packages/imblearn/over_sampling/_smote/base.py:345: FutureWarning: The parameter `n_jobs` has been deprecated in 0.10 and will be removed in 0.12. You can pass an nearest neighbors estimator where `n_jobs` is already set instead.
  warnings.warn(


best loss 1.7254221439361572 {'Overall Survival (4 Years)': {'accuracy': -1, 'mse': 0.11696302, 'auc': 0.48653846153846153}, 'FT': {'accuracy': -1, 'mse': 0.16645414, 'auc': 0.6650219298245614}, 'Aspiration rate Post-therapy': {'accuracy': -1, 'mse': 0.12776816, 'auc': 0.8080128205128205}, 'LRC': {'accuracy': -1, 'mse': 0.11535225, 'auc': 0.5180661577608143}}


/data/DigitalTwinVis/python/Preprocessing.py:192: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/data/DigitalTwinVis/python/Preprocessing.py:192: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/data/DigitalTwinVis/python/Preprocessing.py:252: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_file)


...

best loss 1.6832016706466675 {'Overall Survival (4 Years)': {'accuracy': -1, 'mse': 0.116363265, 'auc': 0.6405926232905097}, 'FT': {'accuracy': -1, 'mse': 0.1416456, 'auc': 0.7491379310344828}, 'Aspiration rate Post-therapy': {'accuracy': -1, 'mse': 0.13230063, 'auc': 0.7184021473119129}, 'LRC': {'accuracy': -1, 'mse': 0.11725632, 'auc': 0.560546875}}
model_done 21


/data/DigitalTwinVis/python/Preprocessing.py:192: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/data/DigitalTwinVis/python/Preprocessing.py:192: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/data/DigitalTwinVis/python/Preprocessing.py:252: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_file)


best loss 2.7489023208618164 {'Overall Survival (4 Years)': {'accuracy': -1, 'mse': 0.13348457, 'auc': 0.575609756097561}, 'FT': {'accuracy': -1, 'mse': 0.14970909, 'auc': 0.6749206349206349}, 'Aspiration rate Post-therapy': {'accuracy': -1, 'mse': 0.13783653, 'auc': 0.7464729593550554}, 'LRC': {'accuracy': -1, 'mse': 0.11642885, 'auc': 0.5976771196283391}}
model_done 22
done with 0 errors


/data/DigitalTwinVis/python/Preprocessing.py:192: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/data/DigitalTwinVis/python/Preprocessing.py:192: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/data/DigitalTwinVis/python/Preprocessing.py:252: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_file)


{'predictions': tensor([[0.7088, 0.4199, 0.3083, 0.7453]], device='cuda:0',
        grad_fn=<MeanBackward1>),
 '5%': tensor([[0.5824, 0.2423, 0.2372, 0.7104]], device='cuda:0',
        grad_fn=<ViewBackward0>),
 '95%': tensor([[0.8144, 0.5752, 0.4743, 0.9146]], device='cuda:0',
        grad_fn=<ViewBackward0>)}

In [17]:
emodel3

EndpointEnsemble(
  (base_models): ModuleList(
    (0-1): 2 x EndpointSimulator(
      (input_dropout): Dropout(p=0, inplace=False)
      (layers): ModuleList(
        (0): Linear(in_features=76, out_features=50, bias=True)
        (1): ReLU()
      )
      (batchnorm): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (dropout): Dropout(p=0.5, inplace=False)
      (relu): Softplus(beta=1, threshold=20)
      (sigmoid): Sigmoid()
      (softmax): LogSoftmax(dim=1)
      (outcome_layer): Linear(in_features=50, out_features=4, bias=True)
    )
  )
  (error_models): ModuleList(
    (0-19): 20 x EndpointSimulator(
      (input_dropout): Dropout(p=0, inplace=False)
      (layers): ModuleList(
        (0): Linear(in_features=76, out_features=50, bias=True)
        (1): ReLU()
      )
      (batchnorm): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (dropout): Dropout(p=0.5, inplace=False)
      (relu): Softplus(beta=1

In [18]:
def gridsearch_attention_transition_models(state=1,attentions=[False]):
    hiddenlayers = [[10],[50],[100],[500],[1000],[10,10],[50,50],[100,100],[500,500],[1000,1000]]
    model_arglist = [{'hidden_layers': h} for h in hiddenlayers]
    best_loss = 100000000000
    best_metrics = {}
    best_args = {}
    best_model = None
    k = 0
    for margs in model_arglist:
        args = {k:v for k,v in margs.items()}
        for use_smote in [True,False]:
            for dropout in [0,.1,.25,.5,.9]:
                args['dropout'] = dropout
                for input_dropout in [0,.1,.25,.5]:
                    args['input_dropout'] = input_dropout
                    model,m_loss,m_metrics = train_state(
                        model_args=args,
                        state=state,
                        use_smote=use_smote,
                        use_attention=False,
                        verbose=False)
                    print('done',k,m_loss)
                    k+=1
                    if m_loss < best_loss:
                        best_loss = m_loss
                        best_metrics  = m_metrics
                        best_model = model
                        best_args = {k:v for k,v in args.items()}
                        best_args['smote'] = use_smote
                        print('_++++++++++New Best++++____')
                        print(best_loss)
                        print(best_metrics)
                        print(best_args)
                        print(use_smote)
                        print('___________')
                        print('++++++++')
                        print()
    print('_________')
    print('+++++++++++')
    print('best stuff',best_loss)
    print(best_metrics)
    print(best_args)
    return best_model
# model1 = gridsearch_attention_transition_models(1)
# model1

In [19]:
# model2 = gridsearch_attention_transition_models(2)
# model2

In [20]:
# model3 = gridsearch_attention_transition_models(3)
# model3

In [21]:
# def load_trained_models():
#     files = Const.tuned_transition_models
#     decision_file = Const.tuned_decision_model
#     [model1,model2,model3] = [torch.load(file) for file in files]
#     decision_model = torch.load(decision_file)
#     return decision_model, model1,model2,model3
# _, model1, model2, model3 =load_trained_models()

In [22]:

def shuffle_col(v,col=None):
    if col is None:
        col = np.random.choice([i for i in range(v.shape[1])])
    idx = torch.randperm(v.shape[0])
    vv = torch.clone(v)
    vv[:,col] = vv[idx,col]
    return vv
    
def train_decision_model(
    tmodel1,
    tmodel2,
    tmodel3,
    use_default_split=True,
    use_bagging_split=False,
    lr=.0001,
    epochs=10000,
    patience=100,
    weights=[1,1,1,1], #realtive weight of survival, feeding tube, aspiration, andl lrc
    imitation_weights=[.5,1,1],#weights of imitation decisions, because ic overtrains too quickly
    imitation_weight=0.1,
    shufflecol_chance = 0.1,
    reward_weight=1,
    split=.7,
    resample_training=False,
    save_path='../data/models/',
    file_suffix='',
    use_attention=True,
    verbose=True,
    threshold_decisions=False,#convert decisiosn to binary in simulation, usually breaks it
    use_smote=False,
    validate_with_memory=True,
    **model_kwargs,
):
    tmodel1.eval()
    tmodel2.eval()
    tmodel3.eval()

    train_ids, test_ids = get_tt_split(use_default_split=use_default_split,use_bagging_split=use_bagging_split,resample_training=resample_training)
    true_ids = train_ids + test_ids #for saving memory without upsampling
    if use_smote:
        dataset = DTDataset(use_smote=True,smote_ids = train_ids)
        train_ids = [i for i in dataset.processed_df.index.values if i not in test_ids]
    else:
        dataset = DTDataset()
    data = dataset.processed_df.copy()
    
    def get_dlt(state):
        if state == 2:
            return data[Const.dlt2].copy()
        d = data[Const.dlt1].copy()
        if state < 1:
            d.values[:,:] = 0
        return d
    
    def get_pd(state):
        if state == 2:
            return data[Const.primary_disease_states2].copy()
        d = data[Const.primary_disease_states].copy()
        if state < 1:
            d.values[:,:] = 0
        return d
    
    def get_nd(state):
        if state == 2:
            return data[Const.nodal_disease_states2].copy()
        d = data[Const.nodal_disease_states].copy()
        if state < 1:
            d.values[:,:] = 0
        return d
    
    def get_cc(state):
        res = data[Const.ccs].copy()
        if state == 1:
            res.values[:,:] = np.zeros(res.values.shape)
        return res
    
    def get_mod(state):
        res = data[Const.modifications].copy()
        #this should have an ic condition but we don't use it anumore anywa
        return res
        
    outcomedf = data[Const.outcomes]
    baseline = dataset.get_state('baseline')
    
    def formatdf(d,dids=train_ids):
        d = df_to_torch(d.loc[dids]).to(model.get_device())
        return d
    
    def makegrad(v):
        if not v.requires_grad:
            v.requires_grad=True
        return v
    
    if use_attention:
        model = DecisionAttentionModel(baseline.shape[1],**model_kwargs)
    else:
        model_kwargs = {k:v for k,v in model_kwargs.items() if 'attention' not in k and 'embed' not in k}
        model = DecisionModel(baseline.shape[1],**model_kwargs)

    hashcode = str(hash(','.join([str(i) for i in train_ids])))
    
    save_file = save_path + 'model_' + model.identifier +'_hash' + hashcode + file_suffix + '.tar'
    model.fit_normalizer(df_to_torch(baseline.loc[train_ids]))
    optimizer = torch.optim.Adam(model.parameters(),lr=lr)

    def outcome_loss(ypred):
        #convert survival to death
        l = torch.mul(torch.mean(-1*(ypred[:,0] - 1)),weights[0])
        for i,weight in enumerate(weights[1:]):
            #weights with negative values will invert the outcome so e.g. Regional control becomes no regional control
            #so the penaly is correct
            if weight > 0:
                newloss = torch.mean(ypred[:,i])*weight
            else:
                newloss = torch.mul(torch.mean(-1*(ypred[:,0] - 1)),-weights[i])
            l = torch.add(l,torch.mul(newloss,weight))
        return l
    
    mse = torch.nn.MSELoss()
    nllloss = torch.nn.NLLLoss()
    bce = torch.nn.BCELoss()
    
    def compare_decisions(d1,d2,d3,ids):
#         ypred = np.concatenate([dd.cpu().detach().numpy().reshape(-1,1) for dd in [d1,d2,d3]],axis=1)
        ytrue = df_to_torch(outcomedf.loc[ids])
        dloss = bce(d1.view(-1),ytrue[:,0])
        dloss += bce(d2.view(-1),ytrue[:,1])
        dloss += bce(d3,view(-1),ytrue[:,2])
        return dloss
        
    def remove_decisions(df):
        cols = [c for c in df.columns if c not in Const.decisions ]
        ddf = df[cols]
        return ddf
    
    makeinput = lambda step,dids: df_to_torch(remove_decisions(dataset.get_input_state(step=step,ids=dids)))
    threshold = lambda x: torch.gt(x,.5).type(torch.FloatTensor)

    
    #save the inputs from the whole dataset for future reference
    if use_attention:
        full_data = []
        for mstep in [0,1,2]:
            full_data_step = [baseline, get_dlt(min(mstep,1)),
                         get_dlt(mstep),get_pd(mstep),get_nd(mstep),get_cc(mstep),get_mod(mstep)]
            full_data_step = torch.cat([formatdf(fd,true_ids) for fd in full_data_step],axis=1)
            full_data.append(full_data_step)
        full_data = torch.stack(full_data)
        model.save_memory(full_data)
        print(full_data.shape)
    def step(train=True):
        if train:
            model.train(True)
            tmodel1.train(True)
            tmodel2.train(True)
            tmodel3.train(True)
            optimizer.zero_grad()
            ids = train_ids
        else:
            ids = test_ids
            model.eval()
            tmodel1.eval()
            tmodel2.eval()
            tmodel3.eval()
            
            
        ytrain = df_to_torch(outcomedf.loc[ids])
        #imitation losses and decision 1
        xxtrained = [baseline, get_dlt(0),get_dlt(0),get_pd(0),get_nd(0),get_cc(0),get_mod(0)]
        xxtrain = [formatdf(xx,ids) for xx in xxtrained]
        
        use_memory = (not train) and validate_with_memory
        if use_attention:
            o1 = model(torch.cat(xxtrain,axis=1),position=0,use_saved_memory = use_memory)
        else:
            o1 = model(torch.cat(xxtrain,axis=1),position=0)
        decision1_imitation = o1[:,3]
        decision1 = o1[:,0]
        if threshold_decisions:
            decision1 = threshold(decision1)

        imitation_loss1 = bce(decision1_imitation,ytrain[:,0])
        imitation_loss1 = torch.mul(imitation_loss1,imitation_weights[0])
        
        x1_imitation = [baseline, get_dlt(1),get_dlt(0),get_pd(1),get_nd(1),get_cc(1),get_mod(1)]
        x1_imitation = [formatdf(xx1,ids) for xx1 in x1_imitation]
        
        if use_attention:
            decision2_imitation = model(torch.cat(x1_imitation,axis=1),position=1,use_saved_memory = use_memory)[:,4]
        else:
            decision2_imitation = model(torch.cat(x1_imitation,axis=1),position=1)[:,4]
            
        imitation_loss2 =  bce(decision2_imitation,ytrain[:,1])
        imitation_loss2 = torch.mul(imitation_loss2,imitation_weights[1])
        
        x2_imitation = [baseline, get_dlt(1),get_dlt(2),get_pd(2),get_nd(2),get_cc(2),get_mod(2)]
        
        x2_imitation = [formatdf(xx2,ids) for xx2 in x2_imitation]
        
        if use_attention:
            decision3_imitation = model(torch.cat(x2_imitation,axis=1),position=2,use_saved_memory = use_memory)[:,5]
        else:
            decision3_imitation = model(torch.cat(x2_imitation,axis=1),position=2)[:,5]
        imitation_loss3 = bce(decision3_imitation,ytrain[:,2])
        imitation_loss3 = torch.mul(imitation_loss3,imitation_weights[2])
        
        #reward decisions
        xx1 = makeinput(1,ids)
        xx2 = makeinput(2,ids)
        xx3 = makeinput(3,ids)

        xx1 = makegrad(xx1)
        xx2 = makegrad(xx2)
        xx3  = makegrad(xx3)
        baseline_train_base = formatdf(baseline,ids)
            
        baseline_train = torch.clone(baseline_train_base)

        
        xi1 = torch.cat([xx1,decision1.view(-1,1)],axis=1)
        [ypd1, ynd1, ymod, ydlt1] = tmodel1(xi1)['predictions']
        
        d1_thresh = torch.gt(decision1.view(-1,1),.5).to(ypd1.device)
        d1_scale = torch.cat([d1_thresh,d1_thresh,torch.ones(d1_thresh.view(-1,1).shape).to(ypd1.device)],dim=1)
        ypd1= torch.mul(ypd1,d1_scale)
        ynd1= torch.mul(ynd1,d1_scale)
        
        x1 = [baseline_train,ydlt1,formatdf(get_dlt(0),ids),ypd1,ynd1,formatdf(get_cc(1),ids),ymod]
        x1=[xx1.to(model.get_device()) for xx1 in x1]
        if use_attention:
            decision2 = model(torch.cat(x1,axis=1),position=1,use_saved_memory = use_memory)[:,1] 
        else:
            decision2 = model(torch.cat(x1,axis=1),position=1)[:,1] 
        if threshold_decisions:
            decision2 = threshold(decision2)
            
        xi2 = torch.cat([xx2,decision1.view(-1,1),decision2.view(-1,1)],axis=1)
        [ypd2,ynd2,ycc,ydlt2] = tmodel2(xi2)['predictions']

        x2 = [baseline_train,ydlt1,ydlt2,ypd2,ynd2,ycc,ymod]
        x2 = [xx2.to(model.get_device()) for xx2 in x2]
        if use_attention:
            decision3 = model(torch.cat(x2,axis=1),position=2,use_saved_memory = use_memory)[:,2]
        else:
            decision3 = model(torch.cat(x2,axis=1),position=2)[:,2]
        if threshold_decisions:
            decision3 = threshold(decision3)
            

        xi3 = torch.cat([xx3,decision1.view(-1,1),decision2.view(-1,1),decision3.view(-1,1)],axis=1)
        outcomes = tmodel3(xi3)['predictions']

        if not train and verbose:
            print(torch.mean(outcomes,dim=0))
        reward_loss = outcome_loss(outcomes)

        loss = torch.add(imitation_loss1,imitation_loss2)
        loss = torch.add(loss,imitation_loss3)
        loss = torch.mul(loss,imitation_weight/3)
        loss = torch.add(loss,torch.mul(reward_loss,reward_weight))
        losses = [imitation_loss1+imitation_loss2+imitation_loss3,reward_loss]
        
        if not train:
            distributions = [torch.mean(torch.gt(d,.5).type(torch.FloatTensor)) for d in [decision1,decision2,decision3]]
            if verbose:
                print('decisions',distributions)

        if train:
            loss.backward()
            optimizer.step()
            return losses
        else:
            scores = []
            for i,decision in enumerate([decision1_imitation,decision2_imitation,decision3_imitation]):
                dec = decision.cpu().detach().numpy()
                dec0 = (dec > .5).astype(int)
                out = ytrain[:,i].cpu().detach().numpy()
                acc = accuracy_score(out,dec > .5)
                auc = roc_auc_score(out,dec)
                scores.append({'decision': i,'accuracy': acc,'auc': auc})
            return losses, scores, distributions
        
    best_val_loss = torch.tensor(1000000000.0)
    steps_since_improvement = 0
    best_val_score = {}
    
    for epoch in range(epochs):
        losses = step(True)
        val_losses,val_metrics,val_distributions = step(False)

        aucsum = np.sum([d['auc'] for d in val_metrics])
        vl = val_losses[1] - aucsum
        if verbose:
            print('______epoch',str(epoch),'_____')
            print('val reward',val_losses[1].item())
            print('imitation reward', val_losses[0].item())
            print('distributions',val_distributions)
            print(val_metrics)
        if vl < best_val_loss:
            best_val_loss = vl
            best_val_score = val_metrics
            best_val_distributions = val_distributions
            steps_since_improvement = 0
            torch.save(model.state_dict(),save_file)
        else:
            steps_since_improvement += 1
        if steps_since_improvement > patience:
            break
    print('++++++++++Final+++++++++++')
    print('best',best_val_loss)
    print(best_val_score)
    model.load_state_dict(torch.load(save_file))
    model.eval()
    return model, best_val_score, best_val_loss, best_val_distributions

from Models import *
args = {
    'hidden_layers': [50,50], 
    'attention_heads': [1,1], 
    'embed_size': 120, 
    'dropout': 0.5, 
    'input_dropout': 0, 
    'shufflecol_chance':  .5,
}

#1.8424
decision_model, _, _, _ = train_decision_model(
    emodel1,emodel2,emodel3,
    lr=.0001,
    use_attention=True,
    imitation_weight=1,
    reward_weight=1,
    weights=[1,1,1,0],
    threshold_decisions=False,
    validate_with_memory=True,
    use_smote=False,
    **args)

/data/DigitalTwinVis/python/Preprocessing.py:192: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/data/DigitalTwinVis/python/Preprocessing.py:252: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_file)


torch.Size([3, 536, 88])
tensor([0.7525, 0.2157, 0.2270, 0.7361], device='cuda:0',
       grad_fn=<MeanBackward1>)
decisions [tensor(0.0753), tensor(0.1986), tensor(0.5274)]
______epoch 0 _____
val reward 1.4427533149719238
imitation reward 1.6186013221740723
distributions [tensor(0.0753), tensor(0.1986), tensor(0.5274)]
[{'decision': 0, 'accuracy': 0.541095890410959, 'auc': 0.6057692307692307}, {'decision': 1, 'accuracy': 0.5958904109589042, 'auc': 0.4602521929824561}, {'decision': 2, 'accuracy': 0.7191780821917808, 'auc': 0.42948717948717946}]
tensor([0.7526, 0.2155, 0.2271, 0.7359], device='cuda:0',
       grad_fn=<MeanBackward1>)
decisions [tensor(0.0822), tensor(0.1986), tensor(0.5342)]
______epoch 1 _____
val reward 1.442632794380188
imitation reward 1.5971204042434692
distributions [tensor(0.0822), tensor(0.1986), tensor(0.5342)]
[{'decision': 0, 'accuracy': 0.5547945205479452, 'auc': 0.6096153846153846}, {'decision': 1, 'accuracy': 0.5958904109589042, 'auc': 0.4651864035087719}

tensor([0.7543, 0.2130, 0.2279, 0.7335], device='cuda:0',
       grad_fn=<MeanBackward1>)
decisions [tensor(0.2534), tensor(0.1849), tensor(0.7260)]
______epoch 16 _____
val reward 1.4409079551696777
imitation reward 1.3764991760253906
distributions [tensor(0.2534), tensor(0.1849), tensor(0.7260)]
[{'decision': 0, 'accuracy': 0.6506849315068494, 'auc': 0.6091346153846153}, {'decision': 1, 'accuracy': 0.7534246575342466, 'auc': 0.5466008771929824}, {'decision': 2, 'accuracy': 0.821917808219178, 'auc': 0.4951923076923077}]
tensor([0.7545, 0.2128, 0.2279, 0.7334], device='cuda:0',
       grad_fn=<MeanBackward1>)
decisions [tensor(0.2671), tensor(0.1849), tensor(0.7329)]
______epoch 17 _____
val reward 1.4407747983932495
imitation reward 1.366870641708374
distributions [tensor(0.2671), tensor(0.1849), tensor(0.7329)]
[{'decision': 0, 'accuracy': 0.6643835616438356, 'auc': 0.6057692307692308}, {'decision': 1, 'accuracy': 0.7534246575342466, 'auc': 0.5493421052631579}, {'decision': 2, 'accur

tensor([0.7560, 0.2113, 0.2273, 0.7331], device='cuda:0',
       grad_fn=<MeanBackward1>)
decisions [tensor(0.5342), tensor(0.2260), tensor(0.8425)]
______epoch 32 _____
val reward 1.4386262893676758
imitation reward 1.2627758979797363
distributions [tensor(0.5342), tensor(0.2260), tensor(0.8425)]
[{'decision': 0, 'accuracy': 0.8493150684931506, 'auc': 0.6000000000000001}, {'decision': 1, 'accuracy': 0.7808219178082192, 'auc': 0.5948464912280702}, {'decision': 2, 'accuracy': 0.821917808219178, 'auc': 0.5570512820512821}]
tensor([0.7561, 0.2112, 0.2272, 0.7331], device='cuda:0',
       grad_fn=<MeanBackward1>)
decisions [tensor(0.5479), tensor(0.2192), tensor(0.8425)]
______epoch 33 _____
val reward 1.438461422920227
imitation reward 1.2579827308654785
distributions [tensor(0.5479), tensor(0.2192), tensor(0.8425)]
[{'decision': 0, 'accuracy': 0.8493150684931506, 'auc': 0.5966346153846155}, {'decision': 1, 'accuracy': 0.7808219178082192, 'auc': 0.5970394736842106}, {'decision': 2, 'accur

tensor([0.7573, 0.2102, 0.2252, 0.7346], device='cuda:0',
       grad_fn=<MeanBackward1>)
decisions [tensor(0.7603), tensor(0.1712), tensor(0.8562)]
______epoch 48 _____
val reward 1.4353885650634766
imitation reward 1.2059096097946167
distributions [tensor(0.7603), tensor(0.1712), tensor(0.8562)]
[{'decision': 0, 'accuracy': 0.8561643835616438, 'auc': 0.5966346153846155}, {'decision': 1, 'accuracy': 0.7808219178082192, 'auc': 0.6241776315789473}, {'decision': 2, 'accuracy': 0.821917808219178, 'auc': 0.6141025641025641}]
tensor([0.7574, 0.2101, 0.2251, 0.7347], device='cuda:0',
       grad_fn=<MeanBackward1>)
decisions [tensor(0.7740), tensor(0.1781), tensor(0.8562)]
______epoch 49 _____
val reward 1.435190200805664
imitation reward 1.2034962177276611
distributions [tensor(0.7740), tensor(0.1781), tensor(0.8562)]
[{'decision': 0, 'accuracy': 0.8561643835616438, 'auc': 0.5961538461538461}, {'decision': 1, 'accuracy': 0.7808219178082192, 'auc': 0.6250000000000001}, {'decision': 2, 'accur

...

tensor([0.7622, 0.2035, 0.1985, 0.7606], device='cuda:0',
       grad_fn=<MeanBackward1>)
decisions [tensor(1.), tensor(0.), tensor(0.0068)]
______epoch 389 _____
val reward 1.4019678831100464
imitation reward 1.3432108163833618
distributions [tensor(1.), tensor(0.), tensor(0.0068)]
[{'decision': 0, 'accuracy': 0.8561643835616438, 'auc': 0.5365384615384615}, {'decision': 1, 'accuracy': 0.7602739726027398, 'auc': 0.678453947368421}, {'decision': 2, 'accuracy': 0.8082191780821918, 'auc': 0.8086538461538462}]
tensor([0.7622, 0.2034, 0.1984, 0.7608], device='cuda:0',
       grad_fn=<MeanBackward1>)
decisions [tensor(1.), tensor(0.), tensor(0.0068)]
______epoch 390 _____
val reward 1.4018045663833618
imitation reward 1.3448448181152344
distributions [tensor(1.), tensor(0.), tensor(0.0068)]
[{'decision': 0, 'accuracy': 0.8561643835616438, 'auc': 0.5384615384615384}, {'decision': 1, 'accuracy': 0.7534246575342466, 'auc': 0.6773574561403509}, {'decision': 2, 'accuracy': 0.8082191780821918, 'au

tensor([0.7625, 0.2031, 0.1976, 0.7616], device='cuda:0',
       grad_fn=<MeanBackward1>)
decisions [tensor(1.), tensor(0.), tensor(0.0068)]
______epoch 406 _____
val reward 1.4006733894348145
imitation reward 1.3800158500671387
distributions [tensor(1.), tensor(0.), tensor(0.0068)]
[{'decision': 0, 'accuracy': 0.8561643835616438, 'auc': 0.5182692307692308}, {'decision': 1, 'accuracy': 0.7602739726027398, 'auc': 0.6779057017543859}, {'decision': 2, 'accuracy': 0.7945205479452054, 'auc': 0.8070512820512821}]
tensor([0.7624, 0.2031, 0.1975, 0.7616], device='cuda:0',
       grad_fn=<MeanBackward1>)
decisions [tensor(1.), tensor(0.), tensor(0.0068)]
______epoch 407 _____
val reward 1.4006354808807373
imitation reward 1.3809384107589722
distributions [tensor(1.), tensor(0.), tensor(0.0068)]
[{'decision': 0, 'accuracy': 0.8561643835616438, 'auc': 0.5173076923076924}, {'decision': 1, 'accuracy': 0.7602739726027398, 'auc': 0.6800986842105263}, {'decision': 2, 'accuracy': 0.7945205479452054, 'a

tensor([0.7627, 0.2029, 0.1970, 0.7622], device='cuda:0',
       grad_fn=<MeanBackward1>)
decisions [tensor(1.), tensor(0.), tensor(0.)]
______epoch 423 _____
val reward 1.399868130683899
imitation reward 1.4146677255630493
distributions [tensor(1.), tensor(0.), tensor(0.)]
[{'decision': 0, 'accuracy': 0.8561643835616438, 'auc': 0.514423076923077}, {'decision': 1, 'accuracy': 0.7465753424657534, 'auc': 0.6757127192982456}, {'decision': 2, 'accuracy': 0.8013698630136986, 'auc': 0.803525641025641}]
tensor([0.7628, 0.2029, 0.1970, 0.7622], device='cuda:0',
       grad_fn=<MeanBackward1>)
decisions [tensor(1.), tensor(0.), tensor(0.)]
______epoch 424 _____
val reward 1.399863362312317
imitation reward 1.4158453941345215
distributions [tensor(1.), tensor(0.), tensor(0.)]
[{'decision': 0, 'accuracy': 0.8561643835616438, 'auc': 0.5144230769230769}, {'decision': 1, 'accuracy': 0.7534246575342466, 'auc': 0.6754385964912281}, {'decision': 2, 'accuracy': 0.8013698630136986, 'auc': 0.8022435897435

tensor([0.7630, 0.2027, 0.1961, 0.7632], device='cuda:0',
       grad_fn=<MeanBackward1>)
decisions [tensor(1.), tensor(0.), tensor(0.)]
______epoch 440 _____
val reward 1.398799180984497
imitation reward 1.4566986560821533
distributions [tensor(1.), tensor(0.), tensor(0.)]
[{'decision': 0, 'accuracy': 0.863013698630137, 'auc': 0.5134615384615384}, {'decision': 1, 'accuracy': 0.7534246575342466, 'auc': 0.6737938596491229}, {'decision': 2, 'accuracy': 0.7945205479452054, 'auc': 0.7990384615384616}]
tensor([0.7630, 0.2027, 0.1961, 0.7632], device='cuda:0',
       grad_fn=<MeanBackward1>)
decisions [tensor(1.), tensor(0.), tensor(0.)]
______epoch 441 _____
val reward 1.3987770080566406
imitation reward 1.463833212852478
distributions [tensor(1.), tensor(0.), tensor(0.)]
[{'decision': 0, 'accuracy': 0.863013698630137, 'auc': 0.5129807692307693}, {'decision': 1, 'accuracy': 0.7534246575342466, 'auc': 0.6726973684210527}, {'decision': 2, 'accuracy': 0.7945205479452054, 'auc': 0.7958333333333

tensor([0.7633, 0.2026, 0.1955, 0.7637], device='cuda:0',
       grad_fn=<MeanBackward1>)
decisions [tensor(1.), tensor(0.), tensor(0.)]
______epoch 457 _____
val reward 1.398193359375
imitation reward 1.5100457668304443
distributions [tensor(1.), tensor(0.), tensor(0.)]
[{'decision': 0, 'accuracy': 0.863013698630137, 'auc': 0.5120192307692308}, {'decision': 1, 'accuracy': 0.7465753424657534, 'auc': 0.6683114035087719}, {'decision': 2, 'accuracy': 0.8013698630136986, 'auc': 0.7919871794871794}]
tensor([0.7634, 0.2026, 0.1955, 0.7638], device='cuda:0',
       grad_fn=<MeanBackward1>)
decisions [tensor(1.), tensor(0.), tensor(0.)]
______epoch 458 _____
val reward 1.3980871438980103
imitation reward 1.5082900524139404
distributions [tensor(1.), tensor(0.), tensor(0.)]
[{'decision': 0, 'accuracy': 0.863013698630137, 'auc': 0.5177884615384615}, {'decision': 1, 'accuracy': 0.7397260273972602, 'auc': 0.6696820175438596}, {'decision': 2, 'accuracy': 0.8013698630136986, 'auc': 0.791987179487179

In [24]:
decision_model, _, _, _ = train_decision_model(
    emodel1,emodel2,emodel3,
    lr=.0001,
    use_attention=True,
    imitation_weight=1,
    reward_weight=1,
    weights=[1,1,1,0],
    threshold_decisions=False,
    validate_with_memory=True,
    use_smote=False,
    **args)

/data/DigitalTwinVis/python/Preprocessing.py:192: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/data/DigitalTwinVis/python/Preprocessing.py:252: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_file)


torch.Size([3, 536, 88])
tensor([0.7590, 0.2175, 0.2257, 0.7381], device='cuda:0',
       grad_fn=<MeanBackward1>)
decisions [tensor(0.5411), tensor(0.4589), tensor(0.5890)]
______epoch 0 _____
val reward 1.2175047397613525
imitation reward 1.6681044101715088
distributions [tensor(0.5411), tensor(0.4589), tensor(0.5890)]
[{'decision': 0, 'accuracy': 0.7671232876712328, 'auc': 0.46442307692307694}, {'decision': 1, 'accuracy': 0.726027397260274, 'auc': 0.6345942982456141}, {'decision': 2, 'accuracy': 0.2876712328767123, 'auc': 0.39198717948717954}]
tensor([0.7590, 0.2173, 0.2258, 0.7380], device='cuda:0',
       grad_fn=<MeanBackward1>)
decisions [tensor(0.5548), tensor(0.4658), tensor(0.5959)]
______epoch 1 _____
val reward 1.2173285484313965
imitation reward 1.647045373916626
distributions [tensor(0.5548), tensor(0.4658), tensor(0.5959)]
[{'decision': 0, 'accuracy': 0.7808219178082192, 'auc': 0.4658653846153846}, {'decision': 1, 'accuracy': 0.7397260273972602, 'auc': 0.6348684210526316

tensor([0.7584, 0.2146, 0.2259, 0.7366], device='cuda:0',
       grad_fn=<MeanBackward1>)
decisions [tensor(0.5479), tensor(0.3014), tensor(0.6849)]
______epoch 16 _____
val reward 1.2146238088607788
imitation reward 1.3720688819885254
distributions [tensor(0.5479), tensor(0.3014), tensor(0.6849)]
[{'decision': 0, 'accuracy': 0.8698630136986302, 'auc': 0.4846153846153846}, {'decision': 1, 'accuracy': 0.7534246575342466, 'auc': 0.6271929824561404}, {'decision': 2, 'accuracy': 0.7191780821917808, 'auc': 0.4541666666666667}]
tensor([0.7584, 0.2144, 0.2259, 0.7365], device='cuda:0',
       grad_fn=<MeanBackward1>)
decisions [tensor(0.5548), tensor(0.2877), tensor(0.6849)]
______epoch 17 _____
val reward 1.214447259902954
imitation reward 1.3578014373779297
distributions [tensor(0.5548), tensor(0.2877), tensor(0.6849)]
[{'decision': 0, 'accuracy': 0.8698630136986302, 'auc': 0.48413461538461533}, {'decision': 1, 'accuracy': 0.7602739726027398, 'auc': 0.625}, {'decision': 2, 'accuracy': 0.732

tensor([0.7583, 0.2114, 0.2265, 0.7346], device='cuda:0',
       grad_fn=<MeanBackward1>)
decisions [tensor(0.6233), tensor(0.2123), tensor(0.7671)]
______epoch 32 _____
val reward 1.2114100456237793
imitation reward 1.2274956703186035
distributions [tensor(0.6233), tensor(0.2123), tensor(0.7671)]
[{'decision': 0, 'accuracy': 0.8972602739726028, 'auc': 0.5144230769230769}, {'decision': 1, 'accuracy': 0.7671232876712328, 'auc': 0.5734649122807017}, {'decision': 2, 'accuracy': 0.8013698630136986, 'auc': 0.5240384615384616}]
tensor([0.7583, 0.2112, 0.2265, 0.7344], device='cuda:0',
       grad_fn=<MeanBackward1>)
decisions [tensor(0.6370), tensor(0.1849), tensor(0.7808)]
______epoch 33 _____
val reward 1.2112137079238892
imitation reward 1.223344087600708
distributions [tensor(0.6370), tensor(0.1849), tensor(0.7808)]
[{'decision': 0, 'accuracy': 0.8972602739726028, 'auc': 0.5091346153846155}, {'decision': 1, 'accuracy': 0.7671232876712328, 'auc': 0.5704495614035089}, {'decision': 2, 'accu

tensor([0.7581, 0.2087, 0.2274, 0.7320], device='cuda:0',
       grad_fn=<MeanBackward1>)
decisions [tensor(0.6781), tensor(0.0822), tensor(0.9315)]
______epoch 48 _____
val reward 1.2086511850357056
imitation reward 1.1825989484786987
distributions [tensor(0.6781), tensor(0.0822), tensor(0.9315)]
[{'decision': 0, 'accuracy': 0.8904109589041096, 'auc': 0.5009615384615385}, {'decision': 1, 'accuracy': 0.773972602739726, 'auc': 0.5394736842105263}, {'decision': 2, 'accuracy': 0.815068493150685, 'auc': 0.6189102564102564}]
tensor([0.7582, 0.2085, 0.2274, 0.7319], device='cuda:0',
       grad_fn=<MeanBackward1>)
decisions [tensor(0.6918), tensor(0.0890), tensor(0.9315)]
______epoch 49 _____
val reward 1.2085307836532593
imitation reward 1.1808942556381226
distributions [tensor(0.6918), tensor(0.0890), tensor(0.9315)]
[{'decision': 0, 'accuracy': 0.8904109589041096, 'auc': 0.5024038461538461}, {'decision': 1, 'accuracy': 0.773972602739726, 'auc': 0.5378289473684211}, {'decision': 2, 'accura

...

tensor([0.7578, 0.1923, 0.2279, 0.7224], device='cuda:0',
       grad_fn=<MeanBackward1>)
decisions [tensor(0.9589), tensor(0.0068), tensor(0.9932)]
______epoch 304 _____
val reward 1.1923277378082275
imitation reward 1.2258894443511963
distributions [tensor(0.9589), tensor(0.0068), tensor(0.9932)]
[{'decision': 0, 'accuracy': 0.8904109589041096, 'auc': 0.5846153846153845}, {'decision': 1, 'accuracy': 0.7465753424657534, 'auc': 0.6334978070175439}, {'decision': 2, 'accuracy': 0.8424657534246576, 'auc': 0.794551282051282}]
tensor([0.7578, 0.1923, 0.2279, 0.7224], device='cuda:0',
       grad_fn=<MeanBackward1>)
decisions [tensor(0.9589), tensor(0.0068), tensor(0.9932)]
______epoch 305 _____
val reward 1.192272424697876
imitation reward 1.2288974523544312
distributions [tensor(0.9589), tensor(0.0068), tensor(0.9932)]
[{'decision': 0, 'accuracy': 0.8904109589041096, 'auc': 0.5836538461538461}, {'decision': 1, 'accuracy': 0.7465753424657534, 'auc': 0.6329495614035088}, {'decision': 2, 'acc

tensor([0.7578, 0.1915, 0.2278, 0.7220], device='cuda:0',
       grad_fn=<MeanBackward1>)
decisions [tensor(0.9795), tensor(0.0068), tensor(1.)]
______epoch 320 _____
val reward 1.191454529762268
imitation reward 1.2617384195327759
distributions [tensor(0.9795), tensor(0.0068), tensor(1.)]
[{'decision': 0, 'accuracy': 0.8904109589041096, 'auc': 0.5673076923076923}, {'decision': 1, 'accuracy': 0.7465753424657534, 'auc': 0.6337719298245613}, {'decision': 2, 'accuracy': 0.8356164383561644, 'auc': 0.7919871794871796}]
tensor([0.7578, 0.1914, 0.2278, 0.7220], device='cuda:0',
       grad_fn=<MeanBackward1>)
decisions [tensor(0.9795), tensor(0.0068), tensor(1.)]
______epoch 321 _____
val reward 1.1913962364196777
imitation reward 1.2646470069885254
distributions [tensor(0.9795), tensor(0.0068), tensor(1.)]
[{'decision': 0, 'accuracy': 0.8904109589041096, 'auc': 0.5673076923076923}, {'decision': 1, 'accuracy': 0.7465753424657534, 'auc': 0.6324013157894737}, {'decision': 2, 'accuracy': 0.83561

tensor([0.7578, 0.1909, 0.2276, 0.7220], device='cuda:0',
       grad_fn=<MeanBackward1>)
decisions [tensor(0.9795), tensor(0.0068), tensor(0.9932)]
______epoch 336 _____
val reward 1.19091796875
imitation reward 1.282821774482727
distributions [tensor(0.9795), tensor(0.0068), tensor(0.9932)]
[{'decision': 0, 'accuracy': 0.8835616438356164, 'auc': 0.5649038461538461}, {'decision': 1, 'accuracy': 0.7534246575342466, 'auc': 0.6378837719298245}, {'decision': 2, 'accuracy': 0.8287671232876712, 'auc': 0.7894230769230769}]
tensor([0.7578, 0.1909, 0.2276, 0.7219], device='cuda:0',
       grad_fn=<MeanBackward1>)
decisions [tensor(0.9726), tensor(0.0068), tensor(0.9932)]
______epoch 337 _____
val reward 1.190912127494812
imitation reward 1.2810481786727905
distributions [tensor(0.9726), tensor(0.0068), tensor(0.9932)]
[{'decision': 0, 'accuracy': 0.8835616438356164, 'auc': 0.5644230769230769}, {'decision': 1, 'accuracy': 0.7602739726027398, 'auc': 0.6384320175438597}, {'decision': 2, 'accuracy

tensor([0.7578, 0.1903, 0.2277, 0.7214], device='cuda:0',
       grad_fn=<MeanBackward1>)
decisions [tensor(0.9863), tensor(0.0068), tensor(0.9932)]
______epoch 352 _____
val reward 1.1903241872787476
imitation reward 1.3146570920944214
distributions [tensor(0.9863), tensor(0.0068), tensor(0.9932)]
[{'decision': 0, 'accuracy': 0.8835616438356164, 'auc': 0.5562499999999999}, {'decision': 1, 'accuracy': 0.7602739726027398, 'auc': 0.6378837719298246}, {'decision': 2, 'accuracy': 0.8287671232876712, 'auc': 0.7759615384615385}]
tensor([0.7578, 0.1903, 0.2277, 0.7215], device='cuda:0',
       grad_fn=<MeanBackward1>)
decisions [tensor(0.9863), tensor(0.0068), tensor(0.9932)]
______epoch 353 _____
val reward 1.1902998685836792
imitation reward 1.319564938545227
distributions [tensor(0.9863), tensor(0.0068), tensor(0.9932)]
[{'decision': 0, 'accuracy': 0.8835616438356164, 'auc': 0.5581730769230769}, {'decision': 1, 'accuracy': 0.7534246575342466, 'auc': 0.637609649122807}, {'decision': 2, 'acc

tensor([0.7578, 0.1897, 0.2279, 0.7209], device='cuda:0',
       grad_fn=<MeanBackward1>)
decisions [tensor(0.9863), tensor(0.0068), tensor(0.9932)]
______epoch 368 _____
val reward 1.1896814107894897
imitation reward 1.344037413597107
distributions [tensor(0.9863), tensor(0.0068), tensor(0.9932)]
[{'decision': 0, 'accuracy': 0.8835616438356164, 'auc': 0.5504807692307692}, {'decision': 1, 'accuracy': 0.7534246575342466, 'auc': 0.6455592105263157}, {'decision': 2, 'accuracy': 0.821917808219178, 'auc': 0.7682692307692308}]
tensor([0.7578, 0.1898, 0.2278, 0.7210], device='cuda:0',
       grad_fn=<MeanBackward1>)
decisions [tensor(0.9863), tensor(0.0068), tensor(0.9932)]
______epoch 369 _____
val reward 1.189754605293274
imitation reward 1.3377217054367065
distributions [tensor(0.9863), tensor(0.0068), tensor(0.9932)]
[{'decision': 0, 'accuracy': 0.8835616438356164, 'auc': 0.551923076923077}, {'decision': 1, 'accuracy': 0.7465753424657534, 'auc': 0.6480263157894737}, {'decision': 2, 'accur

In [ ]:
def gridsearch_decision_model(m1,m2,m3,weights=[1,1,0,0]):
    model_arglist = [

        {
            'hidden_layers': [100,100],
            'attention_heads': [1,1],
        },
        {
            'hidden_layers': [50,50],
            'attention_heads': [1,1],
        },
        {
            'hidden_layers': [100,100],
            'attention_heads': [2,2],
        },
        {
            'hidden_layers': [300],
            'attention_heads': [3],
        },
        {
            'hidden_layers': [600],
            'attention_heads': [3],
        },
        {
            'hidden_layers': [300,300],
            'attention_heads': [3,3],
        },
        {
            'hidden_layers': [600,600],
            'attention_heads': [3,3],
        },
        {
            'hidden_layers': [1000,100],
            'attention_heads': [1],
        },
        {
            'hidden_layers': [1000,100],
            'attention_heads': [5],
        },
    ]
    best_loss = 100000000000
    best_metrics = {}
    best_args = {}
    best_model = None
    best_record = {}
    k = 0
    records = []
    for margs in model_arglist:
        args = {k:v for k,v in margs.items()}
        for embed_size in [0,120]:
            #embed_size = 0 skips the firt layer that makes the sizes right
            if embed_size == 0 and args['attention_heads'][0] != 1:
                continue
            args['embed_size'] = embed_size
            for dropout in [.5,.75]:
                args['dropout'] = dropout
                for input_dropout in [0,.5]:
                    args['input_dropout'] = input_dropout
                    for shufflecol_chance in [.5]:
                        for use_smote in [True,False]:
                            try:
                                args['shufflecol_chance'] = shufflecol_chance
                                model,m_metrics,m_loss,m_distribution = train_decision_model(m1,m2,m3,
                                                                              use_attention=True,
                                                                              validate_with_memory = True,
                                                                            weights=weights,
                                                                            use_smote=use_smote,
                                                                              verbose=False,
                                                                                             **args)
                                entry = {
                                    'metrics': m_metrics,
                                    'loss': m_loss,
                                    'args': args,
                                    'distribution': m_distribution,
                                    'smote': use_smote
                                }
                                records.append(entry)
                                print('done',k,m_loss)
                                print('curr best',best_loss)
                                k+=1
                                if m_loss < best_loss:
                                    best_loss = m_loss
                                    best_metrics  = m_metrics
                                    best_model = model
                                    best_args = args
                                    best_record = entry
                                    print('_++++++++++New Best++++____')
                                    print(best_loss)
                                    print(best_metrics)
                                    print(best_args)
                                    print('___________')
                                    print('++++++++')
                                    print()
                            except Exception as e:
                                print('error',e,args)
    print('_________')
    print('+++++++++++')
    print('best stuff',best_loss)
    print(best_metrics)
    print(best_args)
    return best_model, records, best_record

# from Models import *
# decision_model,records, bestr = gridsearch_decision_model(model1,model2,model3_smote,weights=[1,1,.1,0])
# records

In [ ]:
decision_model

In [27]:
torch.save(decision_model,'../data/models/final_decision_model_' + decision_model.identifier + '.pt')
print('../data/models/final_decision_model_' + decision_model.identifier + '.pt')

../data/models/final_decision_model_statedecisions_input125_dims50,50_dropout0,0.5.pt


In [ ]:
# torch.save(model1,'../data/models/final_transition1_model_' + emodel1.identifier + '.pt')
# torch.save(model2,'../data/models/final_transition2_model_' + emodel2.identifier + '.pt')
# torch.save(model3,'../data/models/final_outcome_model_' + emodel3.identifier + '.pt')
# print('../data/models/final_transition1_model_' + emodel1.identifier + '.pt')
# print('../data/models/final_transition2_model_' + emodel2.identifier + '.pt')
# print('../data/models/final_outcome_model_' + emodel3.identifier + '.pt')

In [26]:
from Models import *
decision_model.set_device('cpu')
emodel1.set_device('cpu')
emodel2.set_device('cpu')
emodel3.set_device('cpu')
torch.save(decision_model,'../resources/decision_model.pt')
torch.save(emodel1,'../resources/transition1_model.pt')
torch.save(emodel2,'../resources/transition2_model.pt')
torch.save(emodel3,'../resources/outcome_model.pt')

In [ ]:
def get_default_predictions(dm):
    res  = []
    for state in [0,1,2]:
        mem = dm.memory[state]
        mem = torch.median(mem,dim=0)[0].type(torch.FloatTensor)
        val = dm(mem.reshape(1,-1),position=state)
        res.append(val.cpu().detach().numpy())
    return np.stack(res)
get_default_predictions(decision_model)